## Top 100 movies of all time

The idea here is we posed a question: which movie is truely the best movie of all time? We don't want to anser this question just by one particular movie website, or just a few movie critics. Therefore, we incorporated data from several different sourses. Then, we created a master dataset for our analysis of interest. 

We pulled the 100 top movie rating from Totten Tomatos movie review website, and we got audience scores through web-scraping, and created a word cloud of reviews from a famous movie critic Roger Ebert.

In [3]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
# Import the Rotten Tomatoes bestofrt TSV file into a DataFrame
df = pd.read_csv('bestofrt.tsv', sep='\t')

In [3]:
df.head()

,ranking,critic_score,title,number_of_critic_ratings
0,1,99,The Wizard of Oz (1939),110
1,2,100,Citizen Kane (1941),75
2,3,100,The Third Man (1949),77
3,4,99,Get Out (2017),282
4,5,97,Mad Max: Fury Road (2015),370


In [6]:
# [Example] Request to an HTML file
import requests
url = 'https://www.rottentomatoes.com/m/et_the_extraterrestrial'
response = requests.get(url)

In [8]:
# [Example] Then we save the request content to a HTML file
with open('et_the_extraterrestrial.html', mode='wb') as file:
    file.write(response.content)

In [ ]:
# [Example cont] Work with HTML in memory
# Besides saving the content to HTML files, we can work with the response content lived in our computer's memory
from bs4 import beautifulSoup
soup = BeautifulSoup(response.content)

We are going to get the user reviews' information from 100 HTML files using the Beautiful Soup library. And here is the steps:
* Creates an empty list, df_list, to which dictionaries will be appended. This list of dictionaries will eventually be converted to a pandas DataFrame (this is the most efficient way of building a DataFrame row by row).
* Loops through each movie's Rotten Tomatoes HTML file in the rt_html folder.
* Opens each HTML file and passes it into a file handle called file.
* Creates a DataFrame called df by converting df_list using the pd.DataFrame constructor.

In [17]:
# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
folder = 'rt_html'
for movie_html in os.listdir(folder):
    with open(os.path.join(folder, movie_html)) as file:
        # Construct an BeautifulSoup object        
        soup = BeautifulSoup(file, 'lxml')
        
        # Get the title, the audience score, and the number of audience rating from the soup
        title = soup.find('title').contents[0][:-len(' - Rotten Tomatoes')]
#         print(title)
#         break
        audience_score = soup.find('div', class_='audience-score meter').find('span').contents[0][:-1]
#         print(audience_score)
#         break
        num_audience_ratings = soup.find('div', class_="audience-info hidden-xs superPageFontColor")\
                                   .find_all('div')[1].contents[2].strip().replace(',', '')

#         print(num_audience_ratings)
#         break
        
        # Append to list of dictionaries
        df_list.append({'title': title,
                        'audience_score': int(audience_score),
                        'number_of_audience_ratings': int(num_audience_ratings)})

# Create a dataframe from the df_list
df = pd.DataFrame(df_list, columns = ['title', 'audience_score', 'number_of_audience_ratings'])